<a href="https://colab.research.google.com/github/baemsu/faq_code/blob/main/%5BFAQ%5D_%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%B2%B4%ED%81%AC%ED%8F%AC%EC%9D%B8%ED%8A%B8%EB%B0%8F%EB%AA%A8%EB%8D%B8%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**(Q)딥러닝의 최적의 학습모델을 어떻게 저장하나요?**

##[방법]
* **딥러닝**시 최적의 epoch의 Weight를 저장하기 위해서는 checkpoint를 적용하여 학습한다.
* checkpoint를 통해 학습 후 weight를 로딩한다.
* 그리고 나서, 해당 모델을 저장하여 사용한다.


## Notation

```
from tensorflow.keras.callbacks import ModelCheckpoint

# 체크포인트 저장하기
checkpoint_path = "my_checkpoint.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only=True, 
                                                 monitor='val_loss', 
                                                 verbose=1)

hist = model_dl.fit(train_x, 
          train_y,  
          epochs=100,
          validation_data=(val_x,val_y),
          callbacks=[cp_callback])

#체크포인트 호출하기
model_dl.load_weights(checkpoint_path)

#모델 저장하기
model_dl.save("my_model.h")

```




# **(예시) 단순회귀모형을 만들고 스케일러와 모델을 함께 저장한 후, 스케일러와 모델를 호출하여 신규 값을 예측.**

## (1) 일단 단순 회귀를 딥러닝 모델로 만들어 봅니다.

In [6]:
#모델 생성
import numpy as np
import pandas as pd

data = {'x':[10,20,30,40,55,67,76,89,99,100],'y':[110,214,313,456,556,687,789,820,950,1023]}
val_data = {'x':[11,23,35],'y':[115,224,300]}

df = pd.DataFrame(data)
df2 = pd.DataFrame(val_data)


train_x = df.drop(columns = ["y"], axis = 1)
train_y = df["y"]

val_x = df2.drop(columns = ["y"], axis = 1)
val_y = df2["y"]

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [13]:
model_dl = tf.keras.Sequential()
model_dl.add(tf.keras.layers.Dense(32, activation='relu', input_shape=[1]))
model_dl.add(tf.keras.layers.Dense(16, activation='relu'))
model_dl.add(tf.keras.layers.Dense(8, activation='relu'))
model_dl.add(tf.keras.layers.Dense(1))

optimizer = tf.keras.optimizers.RMSprop()
model_dl.compile(optimizer=optimizer,
              loss='mse') 

## (2) 체크포인트와 모델을 저장합니다.

In [ ]:
checkpoint_path = "my_checkpoint.ckpt"

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True, 
                                                 monitor='val_loss', 
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
hist = model_dl.fit(train_x, train_y, epochs=1000, validation_data=(val_x,val_y),callbacks=[cp_callback])

In [ ]:
model_dl.load_weights(checkpoint_path)
model_dl.save("my_model.h")

## (3) model을 호출하여 신규 x가 30 일때 y값을 예측해봅니다.

In [26]:
loaded_model = tf.keras.models.load_model("my_model.h")

In [27]:
# 신규 x를 데이터프레임형태로 만듦니다.
new_x=[90]
df_new_x = pd.DataFrame(new_x)

In [ ]:
new_y = loaded_model.predict(df_new_x)
new_y